# IMPORTING LIBRARIES

In [1]:
import os
import cv2
import pandas as pd
import numpy as np      
import streamlit as st  
import tensorflow as tf 

from PIL import Image                                                             
from tensorflow.keras.preprocessing.image import ImageDataGenerator               
from tensorflow.keras.models import Sequential                                   
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense, Dropout   
from tensorflow.keras import layers,models                                     
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

# DATA LOADING

In [6]:
train_D = r"C:\Users\deepa\Downloads\PYHON\Road Signs classification dataset\DATA"
test_D = r"C:\Users\deepa\Downloads\PYHON\Road Signs classification dataset\TEST"

In [7]:
IMG_SIZE = 64

# LOAD DATA

In [10]:
import os
import cv2
import numpy as np

IMG_SIZE = 128  # or whatever size you need

def load_images_from_folder(folder_path):
    X = []
    Y = []
    class_names = sorted(os.listdir(folder_path))  # ensures consistent label order
    
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder_path, class_name)  # ✅ fixed here
        
        if not os.path.isdir(class_folder):
            continue
        
        for file in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file)
            try:
                img = cv2.imread(file_path)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                Y.append(label)
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")
    
    return np.array(X), np.array(Y)

In [11]:
train_X, train_Y = load_images_from_folder(train_D)
test_X, test_Y = load_images_from_folder(test_D)

# ANALYZE THE DATA

In [12]:
print('Training data shape:', train_X.shape, train_Y.shape)
print('Testing data shape:', test_X.shape, test_Y.shape)

Training data shape: (743, 128, 128, 3) (743,)
Testing data shape: (285, 128, 128, 3) (285,)


# INTERPRETATION:

In [13]:
classes = np.unique(train_Y)
nClasses = len(classes)
print('Total number of outputs: ', nClasses)
print('Output classes: ', classes)

Total number of outputs:  30
Output classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


In [17]:
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

plt.figure(figsize=(6, 6))

# Display the first image in training data
plt.subplot(1, 2, 1)
plt.imshow(train_X[1].squeeze(), cmap='gray')
plt.title(f"Ground Truth: {train_Y[1]}")
plt.axis('off')

# Display the first image in testing data
plt.subplot(1, 2, 2)
plt.imshow(test_X[1].squeeze(), cmap='gray')
plt.title(f"Ground Truth: {test_Y[1]}")
plt.axis('off')

plt.tight_layout()
plt.show()



# DATA PREPROCESSING

In [ ]:
train_X.shape

In [ ]:
train_X = train_X.reshape(-1, 64,64, 3)
test_X = test_X.reshape(-1, 64,64, 3)
train_X.shape, test_X.shape

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255
test_X = test_X / 255

In [ ]:
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category Label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

In [ ]:
from sklearn.model_selection import train_test_split
train_X, valid_X, train_X, valid_X = train_test_split(train_X,
                                                      train_Y,
                                                      test_size = 0.2,
                                                      random_state = 13)

In [ ]:
train_X.shape, valid_X.shape

In [ ]:
import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LeakyReLU
from tensorflow.keras.layers import Input
from tensorflow.keras import Model

In [ ]:
batch_size = 32
epochs = 20
num_classes = 10

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(30, activation='softmax')
])

In [ ]:
model.summary()

# COMPLIE THE DATA

In [ ]:
model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, valid_X, train_Y, valid_Y = train_test_split(
    train_X, train_Y, test_size=0.2, random_state=13
)

print("Training data shape:", train_X.shape, train_Y.shape)
print("Validation data shape:", valid_X.shape, valid_Y.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = 30

train_Y = to_categorical(train_Y, num_classes)
valid_Y = to_categorical(valid_Y, num_classes)
test_Y = to_categorical(test_Y, num_classes)

print("train_Y shape:", train_Y.shape)
print("valid_Y shape:", valid_Y.shape)
print("test_Y shape:", test_Y.shape)

In [ ]:
model_train = model.fit(train_X,
                          train_Y,
                          batch_size = batch_size,
                          epochs = epochs, verbose = 1,
                          validation_data=(valid_X, valid_Y))
model_train

In [ ]:
test_eval = model.evaluate(test_X, test_Y,verbose = 0)
print('test loss:',test_eval[0])
print('test accuracy:',test_eval[1])